In [1]:
# Cell 1: Imports and Setup
import json
import random
from pathlib import Path
from pprint import pprint
import pandas as pd

# Dataset paths
DATASET_DIR = Path("resource/dataset")
SA_SGG_PATH = DATASET_DIR / "generation" / "sa-sgg.jsonl"
MA_SGG_PATH = DATASET_DIR / "generation" / "ma-sgg.jsonl"
SGQA_PATH = DATASET_DIR / "understanding" / "sgqa.jsonl"
SGDS_PATH = DATASET_DIR / "understanding" / "sgds.jsonl"

print("Dataset paths configured.")

Dataset paths configured.


In [2]:
# Cell 2: Helper Functions

def load_jsonl(path):
    """Load a JSONL file into a list of dictionaries."""
    with open(path, 'r') as f:
        return [json.loads(line) for line in f]

def sample_data(data, n=1, seed=None):
    """Randomly sample n items from data."""
    if seed is not None:
        random.seed(seed)
    return random.sample(data, min(n, len(data)))

def display_sample(sample, title="Sample"):
    """Pretty print a sample with a title."""
    print(f"\n{'='*80}")
    print(f"{title}")
    print('='*80)
    pprint(sample, width=100, depth=4)

print("Helper functions loaded.")

Helper functions loaded.


In [3]:
# Load SA-SGG dataset
sa_sgg_data = load_jsonl(SA_SGG_PATH)

print(f"SA-SGG Dataset Statistics:")
print(f"  Total samples: {len(sa_sgg_data)}")
print(f"  Fields: {list(sa_sgg_data[0].keys())}")

# Sample and display
sample = sample_data(sa_sgg_data, n=1)[0]

print("\n" + "="*80)
print("RANDOM SA-SGG SAMPLE")
print("="*80)

print(f"\n[Target Sentence]")
print(f"  {sample['target_sentence']}")

print(f"\n[Context]")
print(f"  {sample['context'][:300]}..." if len(sample.get('context', '')) > 300 else f"  {sample.get('context', '(none)')}")

print(f"\n[Ground Truth Scene Graph(s)]")
for i, graph in enumerate(sample['graphs']):
    print(f"  Graph {i+1} (action_id: {graph['action_id']}):")
    for triplet in graph['triplets']:
        print(f"    {triplet[0]} -> {triplet[1]} -> {triplet[2]}")

print(f"\n[Mandatory Space (allowed vocabulary)]")
print(f"  Verbs: {sample['mandatory_space']['verb'][:8]}...")
print(f"  Objects: {sample['mandatory_space']['object'][:8]}...")
print(f"  Relationships: {sample['mandatory_space']['relationship']}")

SA-SGG Dataset Statistics:
  Total samples: 1188
  Fields: ['data_id', 'doc_index', 'text_part_index', 'context', 'target_sentence', 'graph_id', 'is_repeatable', 'graphs', 'verbs', 'mandatory_space', 'context_graphs']

RANDOM SA-SGG SAMPLE

[Target Sentence]
  Finally, the bike was let go, marking the end of the maintenance session.

[Context]
  The bundle was picked up and securely grasped. Using the other hand, the bundle was opened to reveal its contents. Finally, the bundle was laid down carefully. The process began by picking up the screwdriver. With precision, the fastener on the instrument was secured using the screwdriver. Once the ...

[Ground Truth Scene Graph(s)]
  Graph 1 (action_id: -2):
    person -> verb -> release
    release -> dobj -> motorcycle
    release -> from -> hand1
    release -> from -> hand2

[Mandatory Space (allowed vocabulary)]
  Verbs: ['adjust', 'carry', 'check', 'clean', 'close', 'examine', 'fix', 'hit']...
  Objects: ['person', 'carton', 'drawer', 'g

In [4]:
# Load MA-SGG dataset
ma_sgg_data = load_jsonl(MA_SGG_PATH)

print(f"MA-SGG Dataset Statistics:")
print(f"  Total samples: {len(ma_sgg_data)}")
print(f"  Fields: {list(ma_sgg_data[0].keys())}")

# Find a sample with multiple graphs
multi_graph_samples = [s for s in ma_sgg_data if len(s['graphs']) > 1]
sample = sample_data(multi_graph_samples, n=1)[0]

print("\n" + "="*80)
print("RANDOM MA-SGG SAMPLE (with multiple actions)")
print("="*80)

print(f"\n[Target Sentence]")
print(f"  {sample['target_sentence']}")

print(f"\n[Number of Actions]: {len(sample['graphs'])}")

print(f"\n[Ground Truth Scene Graphs]")
for i, graph in enumerate(sample['graphs']):
    print(f"\n  Scene Graph {i+1} (action_id: {graph['action_id']}):")
    for triplet in graph['triplets']:
        print(f"    {triplet[0]} -> {triplet[1]} -> {triplet[2]}")

MA-SGG Dataset Statistics:
  Total samples: 853
  Fields: ['data_id', 'doc_index', 'text_part_index', 'context', 'target_sentence', 'graph_id', 'is_repeatable', 'graphs', 'verbs', 'mandatory_space', 'context_graphs']

RANDOM MA-SGG SAMPLE (with multiple actions)

[Target Sentence]
  Once cooked, the dough strips were transferred to a dish.

[Number of Actions]: 2

[Ground Truth Scene Graphs]

  Scene Graph 1 (action_id: 22):
    remove -> with -> hand1
    remove -> dobj -> cooked-dough-strips
    remove -> with -> spatula
    person -> verb -> remove

  Scene Graph 2 (action_id: 23):
    place -> with -> hand1
    place -> dobj -> cooked-dough-strips
    place -> on -> plate
    person -> verb -> place


In [5]:
# Load SGQA dataset
sgqa_data = load_jsonl(SGQA_PATH)

total_qa_pairs = sum(len(s['qa_pairs']) for s in sgqa_data)

print(f"SGQA Dataset Statistics:")
print(f"  Total samples: {len(sgqa_data)}")
print(f"  Total QA pairs: {total_qa_pairs}")
print(f"  Avg QA pairs per sample: {total_qa_pairs / len(sgqa_data):.1f}")
print(f"  Fields: {list(sgqa_data[0].keys())}")

# Sample and display
sample = sample_data(sgqa_data, n=1)[0]

print("\n" + "="*80)
print("RANDOM SGQA SAMPLE")
print("="*80)

print(f"\n[Context Graphs] ({len(sample['context_graphs'])} scene graphs)")
for i, graph in enumerate(sample['context_graphs'][:3]):  # Show first 3
    print(f"\n  Graph {i+1}:")
    for triplet in graph[:4]:  # Show first 4 triplets
        print(f"    {triplet[0]} -> {triplet[1]} -> {triplet[2]}")
    if len(graph) > 4:
        print(f"    ... ({len(graph) - 4} more triplets)")
if len(sample['context_graphs']) > 3:
    print(f"\n  ... ({len(sample['context_graphs']) - 3} more graphs)")

print(f"\n[Question-Answer Pairs] ({len(sample['qa_pairs'])} pairs)")
for i, qa in enumerate(sample['qa_pairs']):
    print(f"\n  Q{i+1}: {qa['Q']}")
    print(f"  A{i+1}: {qa['A']}")

SGQA Dataset Statistics:
  Total samples: 100
  Total QA pairs: 500
  Avg QA pairs per sample: 5.0
  Fields: ['data_id', 'doc_index', 'text_part_index', 'context_graphs', 'qa_pairs']

RANDOM SGQA SAMPLE

[Context Graphs] (27 scene graphs)

  Graph 1:
    adjust -> dobj -> fabric
    adjust -> with -> hand1
    adjust -> with -> hand2
    person -> verb -> adjust

  Graph 2:
    fold -> dobj -> fabric
    fold -> with -> hand1
    fold -> with -> hand2
    person -> verb -> fold

  Graph 3:
    pick-up -> dobj -> fabric
    pick-up -> with -> hand1
    pick-up -> with -> hand2
    person -> verb -> pick-up

  ... (24 more graphs)

[Question-Answer Pairs] (5 pairs)

  Q1: What object was repeatedly adjusted before the first ironing action occurred?
  A1: fabric

  Q2: Which tool was consistently used for single-handed pick-up actions throughout the sequence?
  A2: hand1

  Q3: What object was placed on the ironing-board immediately after the first folding action?
  A3: fabric

  Q4: Whic

In [6]:
# Load SGDS dataset
sgds_data = load_jsonl(SGDS_PATH)

print(f"SGDS Dataset Statistics:")
print(f"  Total samples: {len(sgds_data)}")
print(f"  Fields: {list(sgds_data[0].keys())}")

# Sample and display
sample = sample_data(sgds_data, n=1)[0]

print("\n" + "="*80)
print("RANDOM SGDS SAMPLE")
print("="*80)

print(f"\n[Scene Graph Triplets]")
for triplet in sample['triplet']:
    print(f"  {triplet[0]} -> {triplet[1]} -> {triplet[2]}")

print(f"\n[Candidate Descriptions]")
for i, desc in enumerate(sample['variations']):
    marker = " <-- CORRECT" if i == sample['position'] else ""
    print(f"  [{chr(65+i)}] {desc}{marker}")

print(f"\n[Correct Answer]: {chr(65 + sample['position'])} (position {sample['position']})")

SGDS Dataset Statistics:
  Total samples: 250
  Fields: ['target_sentence', 'position', 'variations', 'triplet', 'context_graphs']

RANDOM SGDS SAMPLE

[Scene Graph Triplets]
  person -> verb -> pour
  pour -> dobj -> water
  pour -> from -> cup
  pour -> into -> keg
  pour -> with -> hand1

[Candidate Descriptions]
  [A] Once secured, the fastener was let go.
  [B] The crochet hook was then picked up again.
  [C] Finally, the plant was modified with one hand to achieve the perfect alignment within the container.
  [D] Finally, the water was transferred from the mug into the barrel, ensuring it was ready for use. <-- CORRECT
  [E] Finally, the paint can was put down carefully.

[Correct Answer]: D (position 3)


In [7]:
# Summary table
total_qa = sum(len(s['qa_pairs']) for s in sgqa_data)

summary = pd.DataFrame({
    'Task': ['SA-SGG', 'MA-SGG', 'SGQA', 'SGDS'],
    'Type': ['Generation', 'Generation', 'Understanding', 'Understanding'],
    'Samples': [len(sa_sgg_data), len(ma_sgg_data), len(sgqa_data), len(sgds_data)],
    'Description': [
        'Single-Action Scene Graph Generation',
        'Multi-Action Scene Graph Generation', 
        f'Scene Graph QA ({total_qa} QA pairs)',
        'Scene Graph Description Selection'
    ]
})

print("TSG-Bench Dataset Summary")
print("="*80)
print(summary.to_string(index=False))
print(f"\nTotal samples: {summary['Samples'].sum()}")

TSG-Bench Dataset Summary
  Task          Type  Samples                          Description
SA-SGG    Generation     1188 Single-Action Scene Graph Generation
MA-SGG    Generation      853  Multi-Action Scene Graph Generation
  SGQA Understanding      100        Scene Graph QA (500 QA pairs)
  SGDS Understanding      250    Scene Graph Description Selection

Total samples: 2391
